## Genetic Algorithms ##

*The contents of this file are taken from Classic Computer Science Problems in Python (Chapter 5) Copyright 2018 David Kopec. This is a Jupyter Notebook implementation of the chapter*

A genetic algorithm includes a:

(1) Population of _chromosomes_ containing _genes_ which specify their traits, all competing to solve a problem

(2) A _fitness function_ which determines how well a chromosome solves a problem

(3) Series of *generations* in which fit chromosomes are more likely to be selected to *reproduce*

(4) Probability that in each generation, two chromosomes have their genes merged (*crossover*)

(5) Probability that in each generation, a gene in a chromosome may randomly change (_mutation_)

The algorithm ends either when some individual chromosome's fitness has surpassed a specific threshhold, or when a specified number of generations has been reached

Usually most problems aren't best solved with a Genetic Algorithm but instead have a deterministic solution (remember, the GA relies on three *random* processes: selection, crossover, and mutation). In problems which lack deterministic solutions, they can be a good choice.

### The Chromosome ###

A chromosome should be able to: 

(1) Determine its own fitness 

(2) Create an instance with randomly selected genes (to fill the first generation)

(3) Crossover with another to create children

(4) Mutate to create minor, random alterations


In [5]:
from __future__ import annotations
from typing import TypeVar, Tuple, Type
from abc import ABC, abstractmethod

T = TypeVar('T', bound='Chromosome') # for returning self

# Base class for all chromosomes; all methods must be overridden
class Chromosome(ABC):
    @abstractmethod
    def fitness(self) -> float:
        ...

    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...

    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...

    @abstractmethod
    def mutate(self) -> None:
        ...

### Creating the Genetic Algorithm Class ##

In this very general implementation, a few kew components are left configurable:

(1) How many chromosomes are in our population

(2) The fitness threshold which stops the algorithm

(3) How chromosomes are selected for reproduction

(4) How should crossover be implemented? At what probability?

(5) Probability for mutation

(6) Number of generations

In [7]:
from __future__ import annotations
from typing import TypeVar, Generic, List, Tuple, Callable
from enum import Enum
from random import choices, random
from heapq import nlargest
from statistics import mean

C = TypeVar('C', bound=Chromosome) # type of the chromosomes


class GeneticAlgorithm(Generic[C]):
    # Roulette selection: gives each chromosome a chance of being picked proportionate to fitness
    # Tournmant selection: a specific number of random chromosomes are challenged against eachother, and the one with the best fitness is chosen
    SelectionType = Enum("SelectionType", "ROULETTE TOURNAMENT") 

    def __init__(self, initial_population: List[C], threshold: float, max_generations: int = 100, mutation_chance: float = 0.01, 
                 crossover_chance: float = 0.7, selection_type: SelectionType = SelectionType.TOURNAMENT) -> None:
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
        #we use type() to refer to the specific subclass of Chromosome we are finding the fitness of
        self._fitness_key: Callable = type(self._population[0]).fitness

    # Use the probability distribution wheel to pick 2 parents
    # Note: will not work with negative fitness results
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        return tuple(choices(self._population, weights=wheel, k=2))

    # Choose num_participants at random and take the best 2
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        participants: List[C] = choices(self._population, k=num_participants)
        return tuple(nlargest(2, participants, key=self._fitness_key))

    # Replace the population with a new generation of individuals
    def _reproduce_and_replace(self) -> None:
        '''
        1) Two chromosomes, called parents, are selected for reproduction using one of the two selection methods.
        2) There is _crossover_chance that the two parents will be combined to produce two new chromosomes, in which case they are added to new_population
        3) If new_population has as many chromosomes as _population, it replaces. Otherwise, return to step 1
        '''
        new_population: List[C] = []
        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            # pick the 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                parents: Tuple[C, C] = self._pick_roulette([x.fitness() for x in self._population])
            else:
                parents = self._pick_tournament(len(self._population) // 2)
            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                new_population.extend(parents)
        # if we had an odd number, we'll have 1 extra, so we remove it
        if len(new_population) > len(self._population):
            new_population.pop()
        self._population = new_population # replace reference

    # With _mutation_chance probability mutate each individual
    def _mutate(self) -> None:
        for individual in self._population:
            if random() < self._mutation_chance:
                individual.mutate()

    # Run the genetic algorithm for max_generations iterations
    # and return the best individual found
    def run(self) -> C:
        best: C = max(self._population, key=self._fitness_key)
        for generation in range(self._max_generations):
            # early exit if we beat threshold
            if best.fitness() >= self._threshold:
                return best
            print(f"Generation {generation} Best {best.fitness()} Avg {mean(map(self._fitness_key, self._population))}")
            self._reproduce_and_replace()
            self._mutate()
            highest: C = max(self._population, key=self._fitness_key)
            if highest.fitness() > best.fitness():
                best = highest # found a new best
        return best # best we found in _max_generations


### A naive test ##

Let's try to solve a simple problem with this approach. 

For what values of $x$ and $y$ does the equation $6x-x^2+4y-y^2$ yield the largest number?

In [8]:
from __future__ import annotations
from typing import Tuple, List
from random import randrange, random
from copy import deepcopy


class SimpleEquation(Chromosome):
    def __init__(self, x: int, y: int) -> None:
        self.x: int = x
        self.y: int = y

    def fitness(self) -> float: # 6x - x^2 + 4y - y^2
        return 6 * self.x - self.x * self.x + 4 * self.y - self.y * self.y

    @classmethod
    def random_instance(cls) -> SimpleEquation:
        return SimpleEquation(randrange(100), randrange(100))

    def crossover(self, other: SimpleEquation) -> Tuple[SimpleEquation, SimpleEquation]:
        child1: SimpleEquation = deepcopy(self)
        child2: SimpleEquation = deepcopy(other)
        child1.y = other.y
        child2.y = self.y
        return child1, child2

    def mutate(self) -> None:
        if random() > 0.5: # mutate x
            if random() > 0.5:
                self.x += 1
            else:
                self.x -= 1
        else: # otherwise mutate y
            if random() > 0.5:
                self.y += 1
            else:
                self.y -= 1

    def __str__(self) -> str:
        return f"X: {self.x} Y: {self.y} Fitness: {self.fitness()}"


if __name__ == "__main__":
    initial_population: List[SimpleEquation] = [SimpleEquation.random_instance() for _ in range(20)]
    ga: GeneticAlgorithm[SimpleEquation] = GeneticAlgorithm(initial_population=initial_population, threshold=13.0, max_generations = 100, mutation_chance = 0.1, crossover_chance = 0.7)
    result: SimpleEquation = ga.run()
    print(result)

Generation 0 Best -437 Avg -5383.45
Generation 1 Best -77 Avg -997
Generation 2 Best -77 Avg -162.5
Generation 3 Best -77 Avg -89.1
Generation 4 Best -60 Avg -76.25
Generation 5 Best -55 Avg -70.15
Generation 6 Best -55 Avg -58.25
Generation 7 Best -55 Avg -56.85
Generation 8 Best -55 Avg -55
Generation 9 Best -40 Avg -55.6
Generation 10 Best -40 Avg -46.75
Generation 11 Best -27 Avg -39.85
Generation 12 Best -27 Avg -39.3
Generation 13 Best -27 Avg -32.05
Generation 14 Best -24 Avg -26.85
Generation 15 Best -16 Avg -24.95
Generation 16 Best -16 Avg -18.9
Generation 17 Best -13 Avg -16.65
Generation 18 Best -7 Avg -14.8
Generation 19 Best 0 Avg -11.8
Generation 20 Best 3 Avg -3.95
Generation 21 Best 3 Avg 1.75
Generation 22 Best 8 Avg 2.75
Generation 23 Best 8 Avg 3.35
Generation 24 Best 9 Avg 4
Generation 25 Best 12 Avg 8.75
Generation 26 Best 12 Avg 9.45
Generation 27 Best 12 Avg 10.65
Generation 28 Best 12 Avg 11.95
X: 3 Y: 2 Fitness: 13


An interesting application
https://www.hudsonrivertrading.com/hrtbeat/intern-spotlight-2023-hrt-ai-labs-summer-projects/